In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from imblearn.over_sampling import SMOTE
from chart_utils import TimeSeriesImageDataset, accuracy_fn

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to read UCR dataset
def read_ucr(filename):
    data = []
    labels = []
    
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            if len(parts) < 2:
                continue
            features = [float(f) for f in parts[:-1]]
            label = int(parts[-1].split(':')[-1])
            data.append(features)
            labels.append(label)
    
    print(f"Loaded {len(data)} samples from {filename}")
    return np.array(data), np.array(labels)

# File paths
train_file = 'ECG5000/ECG5000_TRAIN.ts'
test_file = 'ECG5000/ECG5000_TEST.ts'

# Load dataset
x_train, y_train = read_ucr(train_file)
x_test, y_test = read_ucr(test_file)

# Normalize labels to be within range [0, num_classes-1]
unique_labels = np.unique(y_train)
label_map = {label: idx for idx, label in enumerate(unique_labels)}
y_train = np.array([label_map[label] for label in y_train])
y_test = np.array([label_map[label] for label in y_test])

nb_classes = len(unique_labels)

# Normalize features
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

# Apply SMOTE to the training data
smote = SMOTE(k_neighbors = 1)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# Convert to PyTorch tensors
X_train = torch.tensor(x_train_resampled, dtype=torch.float32)
X_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train_resampled, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Define a 2D CNN Model
class Simple2DCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(Simple2DCNN, self).__init__()
        self.layer_stack = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.layer_stack(x)

# Data Augmentation
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to a smaller size for simpler model
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])

# Create data loaders
train_dataset = TimeSeriesImageDataset(X_train.numpy(), y_train.numpy(), transform)
test_dataset = TimeSeriesImageDataset(X_test.numpy(), y_test.numpy(), transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model, criterion, optimizer, and scheduler
model = Simple2DCNN(6, nb_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# Training and evaluation loop
num_epochs = 50
best_test_accuracy = 0
patience = 10
trigger_times = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images_area, images_bar, labels in train_loader:
        images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device)
        
        # Combine area and bar charts along the channel dimension
        combined_images = torch.cat((images_area, images_bar), dim=1)
        
        optimizer.zero_grad()
        outputs = model(combined_images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.2f}%')

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images_area, images_bar, labels in test_loader:
            images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device)
            
            # Combine area and bar charts along the channel dimension
            combined_images = torch.cat((images_area, images_bar), dim=1)
            
            outputs = model(combined_images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = 100 * correct / total
    print(f'Test Accuracy after Epoch {epoch+1}: {test_accuracy:.2f}%')
    scheduler.step(epoch_loss)

    if test_accuracy > best_test_accuracy:
        best_test_accuracy = test_accuracy
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

print(f'Best Test Accuracy: {best_test_accuracy:.2f}%')


Loaded 500 samples from ECG5000/ECG5000_TRAIN.ts
Loaded 4500 samples from ECG5000/ECG5000_TEST.ts
Epoch [1/50], Loss: 1.6686, Training Accuracy: 19.59%
Test Accuracy after Epoch 1: 1.91%


C:\Users\vmadhuu\AppData\Local\anaconda3\envs\tscproj\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch [2/50], Loss: 1.6100, Training Accuracy: 20.00%
Test Accuracy after Epoch 2: 1.91%
Epoch [3/50], Loss: 1.6100, Training Accuracy: 19.79%
Test Accuracy after Epoch 3: 1.91%
Epoch [4/50], Loss: 1.6097, Training Accuracy: 20.00%
Test Accuracy after Epoch 4: 1.91%
Epoch [5/50], Loss: 1.6097, Training Accuracy: 20.00%
Test Accuracy after Epoch 5: 1.91%
Epoch [6/50], Loss: 1.6097, Training Accuracy: 20.00%
Test Accuracy after Epoch 6: 1.91%
Epoch [7/50], Loss: 1.6097, Training Accuracy: 20.00%
Test Accuracy after Epoch 7: 1.91%
Epoch [8/50], Loss: 1.6097, Training Accuracy: 20.00%
Test Accuracy after Epoch 8: 1.91%
Epoch [9/50], Loss: 1.6097, Training Accuracy: 20.00%
Test Accuracy after Epoch 9: 1.91%
Epoch [10/50], Loss: 1.6096, Training Accuracy: 20.00%
Test Accuracy after Epoch 10: 1.91%
Epoch [11/50], Loss: 1.6095, Training Accuracy: 20.00%
Test Accuracy after Epoch 11: 1.91%
Early stopping at epoch 11
Best Test Accuracy: 1.91%
